In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')


In [ ]:
!ls '/content/drive/MyDrive/CS 639 Project/'

In [ ]:
root = '/content/drive/MyDrive/CS 639 Project/'

In [ ]:
# load label of each image
def label_loader(label_file):
    train_img_label_pair = {}
    test_img_label_pair ={}
    with open(label_file) as all_label:
        for label in all_label:
            label = label.rstrip()
            if label.startswith("train"):
                train_img_label_pair[label.split(" ")[0][:-4]+"_aligned.jpg"] = int(label.split(" ")[1])
            if label.startswith("test"):
                test_img_label_pair[label.split(" ")[0][:-4]+"_aligned.jpg"] = int(label.split(" ")[1])
    return train_img_label_pair, test_img_label_pair

In [ ]:
import numpy as np
def plot(history, log_name, num_epoch):
    """
    plot accuracy and loss save to "./imgs"
    :param history: tensorflow History Object
    :param log_name: String:name of the log
    :param num_epoch: int: number of epoches
    """
    import matplotlib.pyplot as plt
    import os
    if not os.path.exists(root + 'graph'):
        print("First run, make dir")
        os.makedirs(root + 'graph')
    plt.plot(np.linspace(1, num_epoch, num_epoch),
             np.array(history.history["categorical_accuracy"]), label='Training Accuracy', color='b')
    plt.plot(np.linspace(1, num_epoch, num_epoch),
             np.array(history.history["val_categorical_accuracy"]), label='Test Accuracy', color='r')
    plt.legend()
    plt.title("Accuracy" + log_name + time.ctime())
    plt.savefig(root +"graph/Accuracy " + log_name)
    # plt.show()
    plt.close()
    plt.plot(np.linspace(1, num_epoch, num_epoch), np.array(history.history["loss"]), label='Training Loss', color='b')
    plt.plot(np.linspace(1, num_epoch, num_epoch),
             np.array(history.history["val_loss"]), label='Test Loss', color='r')
    plt.legend()
    plt.title("Loss" + log_name + time.ctime())
    plt.savefig(root +"graph/Loss " + log_name)
    # plt.show()
    plt.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
def Model():
    model = tf.keras.Sequential([
        layers.Conv2D(128, kernel_size=3, activation='relu', input_shape=(100,100,3)),
        layers.Conv2D(128, kernel_size=3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Flatten(),
        layers.Dense(7, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adadelta(0.001),
                  loss='categorical_crossentropy',
                 metrics=[tf.keras.metrics.categorical_accuracy])

    return model

In [ ]:
# create image label pair as a dictionary
label_file = root + 'list_patition_label.txt'
train_img_label_pair, test_img_label_pair = label_loader(label_file)

# Use pickle to speed up reading data
import pickle
input_shape = (100, 100)
print("Loading Data")
try:
    train_img, train_label, test_img, test_label = pickle.load(open(root + "processed_data_"+str(input_shape)+".pickle", 'rb'))
    print("Shape", train_img.shape)
except:
    print("Error occurred when loading data")

import os
trained_model_dir = root + "trained_model_optimized"
if not os.path.exists(trained_model_dir):
    print("First run, make model dir")
    os.makedirs(trained_model_dir)

In [ ]:
num_epoch = 200
batch_size = 64
model = Model()

log_name = input("What's the name of this run?:")

# Keras model and Tensor Board
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import time
tb = TensorBoard(log_dir="logs/" + log_name)
chk = ModelCheckpoint(trained_model_dir+log_name+"_chk.h5", monitor='val_categorical_accuracy', verbose=0,
                      save_best_only=True, save_weights_only=True, mode='max', save_freq=1)

history = model.fit(train_img, train_label, epochs=num_epoch, batch_size=batch_size,
                    validation_data=(test_img, test_label), callbacks=[tb, chk])

model.summary()
plot(history, log_name, num_epoch)
model.save(trained_model_dir+"/"+log_name+".h5")
loss, accuracy = model.evaluate(test_img, test_label,batch_size=32)
print("test loss:", loss)
print("test accuracy:", accuracy)
